In [41]:
#Deprecated

0.0066995390717118665


In [1]:
%run "U1. Imports.ipynb"
%run "U2. Utilities.ipynb"
%run "U3. Classes.ipynb"
# %run "D3. Simulation Functions.ipynb"

baseball_path = r'C:\Users\james\Documents\MLB\Database'

db_path = r'C:\Users\james\Documents\MLB\Database\MLBDB.db'
engine = create_engine(f'sqlite:///{db_path}')

In [23]:
%run "A02. MLB API.ipynb"

In [24]:
complete_dataset = create_inputs(start_year=2023, end_year=2023, short=50, long=300)


In [105]:
dp_dataset = complete_dataset[['outs', 'event', 'eventsModel', 'description', 'onFirst', 'onSecond', 'onThird']]

In [106]:
dp_dataset['double_play'] = dp_dataset['description'].str.contains('double play', case=False, na=False).astype('int')

dp_dataset['dp_possible'] = ((dp_dataset['onFirst'] == 1) | (dp_dataset['onSecond'] == 1) | (dp_dataset['onThird'] == 1)) & ((dp_dataset['outs'].astype('int')< 3) | (dp_dataset['double_play'] == 1)).astype('int')


In [107]:
dp_dataset = dp_dataset.query('dp_possible == 1')
dp_dataset = dp_dataset[dp_dataset['eventsModel'].isin(['po', 'go', 'lo', 'fo'])]

In [109]:
odds = dp_dataset.groupby(['onFirst', 'onSecond', 'onThird', 'eventsModel'])['double_play'].mean().reset_index()

In [110]:
odds = odds.pivot_table(index=['onFirst', 'onSecond', 'onThird'], columns='eventsModel', values='eventsModel', aggfunc='sum', fill_value=0)
odds.columns.name = None
odds.reset_index(inplace=True)
odds.rename(columns={'onFirst':'runner_1b', 'onSecond':'runner_2b', 'onThird':'runner_3b', 'fo':'fo_dp', 'go':'go_dp', 'lo':'ld_dp', 'po':'pu_dp'}, inplace=True)

odds.loc[len(odds)] = [0] * len(odds.columns)

# odds.drop(columns={'index'}, inplace=True)

odds

,runner_1b,runner_2b,runner_3b,fo_dp,go_dp,ld_dp,pu_dp
0,0,0,1,0.000000,0.058632,0,0
1,0,1,0,0.000000,0.027027,0,0
2,0,1,1,0.009202,0.064024,0,0
3,1,0,0,0.000000,0.462695,0,0
4,1,0,1,0.019763,0.443769,0,0
5,1,1,0,0.000000,0.450170,0,0
6,1,1,1,0.028571,0.417808,0,0
7,0,0,0,0.000000,0.000000,0,0


In [111]:
odds.to_csv(os.path.join(baseball_path, "Statcast Data", "Double Play Rates.csv"), index=False)